<a href="https://colab.research.google.com/github/DessyVV/llm-experiments/blob/main/Gemini_and_mUSE_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install and load requirements


In [ ]:
%%capture
!pip install -q -U google-generativeai
!pip install tensorflow tensorflow-hub
!pip install tensorflow-text --upgrade

In [ ]:
import google.generativeai as genai
from google.colab import userdata

from langchain.prompts import PromptTemplate

import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text  # This must be imported for SentencepieceOp for the encoder

from sklearn.metrics.pairwise import cosine_similarity

# Set google API key to access gemini-pro
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel('gemini-pro')

## Experiment with gemini-pro with Bulgarian

In [ ]:
# @title #### Summarise with Gemini an extract from "Държавен вестник Брой 1, 3.1.2025 г."
response = model.generate_content("Обобщи кратко и на разбираем език следното постановление: ПОСТАНОВЛЕНИЕ № 493 ОТ 30 ДЕКЕМВРИ 2024 Г. за одобряване на допълнителни разходи по бюджета на Министерството на енергетиката за 2024 г. МИНИСТЕРСКИЯТ СЪВЕТ ПОСТАНОВИ: Чл. 1. (1) Одобрява допълнителни разходи в размер до 2 200 000 лв. по бюджета на Министерството на енергетиката за 2024 г. за финансова подкрепа на крайните битови клиенти на електрическа енергия, останали без електрозахранване в периода от 21 декември 2024 г. – 31 декември 2024 г. включително.  (2) Средствата по ал. 1 да се осигурят чрез преструктуриране на разходите и/или трансферите по централния бюджет за 2024 г. Чл. 2. (1) Със сумата по чл. 1, ал. 1 да се увеличат разходите по бюджета на Министерството на енергетиката за 2024 г. по „Политика в областта на устойчивото и конкурентоспособно енергийно развитие“, бюджетна програма „Сигурност при енергоснабдяването и при управлението на РАО и ИЕЯС“. (2) Със сумата по чл. 1, ал. 1 да се увеличат показателите по чл. 21, ал. 3 от Закона за държавния бюджет на Република България за 2024 г. (3) Икономията по показателите по ал. 2 не може да се използва за поемането на нови ангажименти и нови задължения за разходи за други цели.Чл. 3. (1) За извършване на плащанията по чл. 1, ал. 1 се използва сметка за чужди средства на Министерството на енергетиката.(2) Преведените до 31 декември 2024 г. по сметката по ал. 1 суми се отразяват в изпълнението на съответните разходни показатели по бюджета на Министерството на енергетиката за 2024 г. Чл. 4. Министърът на енергетиката да извърши съответните промени по бюджета на Министерството на енергетиката за 2024 г. и да уведоми министъра на финансите. Чл. 5. Министърът на финансите да извърши съответните промени по централния бюджет за 2024 г. Чл. 6. Възлага на министъра на енергетиката и на министъра на финансите да утвърдят указания за изплащане на целевата финансова подкрепа по чл. 1.")
print(response.text)

Правителството е одобрило допълнително финансиране в размер на 2,2 милиона лева за Министерството на енергетиката за 2024 г. с цел осигуряване на финансова помощ на битови клиенти на електрическа енергия, които са били без електроснабдяване от 21 до 31 декември 2024 г. Средствата ще бъдат осигурени чрез преразпределяне на разходи или трансфери от други части на държавния бюджет.


#### Response from BgGPT using the same prompt

"Министерският съвет одобрява допълнителни разходи в размер до 2 200 000 лв. по бюджета на Министерството на енергетиката за 2024 г. Тези средства ще бъдат използвани за финансова подкрепа на крайните битови клиенти на електрическа енергия, които останат без електрозахранване в периода от 21 декември 2024 г. до 31 декември 2024 г. Средствата ще бъдат осигурени чрез преструктуриране на разходите и/или трансферите по централния бюджет за 2024 г. Разходите ще бъдат увеличени по бюджетната програма „Сигурност при енергоснабдяването и при управлението на РАО и ИЕЯС“. Плащанията ще се извършват чрез сметка за чужди средства на Министерството на енергетиката, а министрите на енергетиката и финансите ще направят необходимите промени по бюджета и ще утвърдят указания за изплащане на целевата финансова подкрепа."

In [ ]:
# @title #### Load 4 examples from "Държавен вестник Брой 1, 3.1.2025 г."
documents = [
    {"text": "ПОСТАНОВЛЕНИЕ № 493 ОТ 30 ДЕКЕМВРИ 2024 Г. за одобряване на допълнителни разходи по бюджета на Министерството на енергетиката за 2024 г. МИНИСТЕРСКИЯТ СЪВЕТ ПОСТАНОВИ: Чл. 1. (1) Одобрява допълнителни разходи в размер до 2 200 000 лв. по бюджета на Министерството на енергетиката за 2024 г. за финансова подкрепа на крайните битови клиенти на електрическа енергия, останали без електрозахранване в периода от 21 декември 2024 г. – 31 декември 2024 г. включително.  (2) Средствата по ал. 1 да се осигурят чрез преструктуриране на разходите и/или трансферите по централния бюджет за 2024 г. Чл. 2. (1) Със сумата по чл. 1, ал. 1 да се увеличат разходите по бюджета на Министерството на енергетиката за 2024 г. по „Политика в областта на устойчивото и конкурентоспособно енергийно развитие“, бюджетна програма „Сигурност при енергоснабдяването и при управлението на РАО и ИЕЯС“. (2) Със сумата по чл. 1, ал. 1 да се увеличат показателите по чл. 21, ал. 3 от Закона за държавния бюджет на Република България за 2024 г. (3) Икономията по показателите по ал. 2 не може да се използва за поемането на нови ангажименти и нови задължения за разходи за други цели.Чл. 3. (1) За извършване на плащанията по чл. 1, ал. 1 се използва сметка за чужди средства на Министерството на енергетиката.(2) Преведените до 31 декември 2024 г. по сметката по ал. 1 суми се отразяват в изпълнението на съответните разходни показатели по бюджета на Министерството на енергетиката за 2024 г. Чл. 4. Министърът на енергетиката да извърши съответните промени по бюджета на Министерството на енергетиката за 2024 г. и да уведоми министъра на финансите. Чл. 5. Министърът на финансите да извърши съответните промени по централния бюджет за 2024 г. Чл. 6. Възлага на министъра на енергетиката и на министъра на финансите да утвърдят указания за изплащане на целевата финансова подкрепа по чл. 1.", "id": 8381},
    {"text": "621. – Висшето училище по застраховане и финанси – София, обявява конкурс за прием на докторанти в редовна форма на обучение по чл. 21, ал. 7 от ЗВО (докторските места се финансират от Програма „Образование“ по проект BG05SFPR001-3.004-0002-C01 „Развитие и усъвършенстване на професионалното и интердисциплинарно образование в техническите и икономически науки и повишаване на научноизследователския капацитет на добивната промишленост чрез проектна докторантура“) в ПН 3.8. Икономика, специалност: „Финанси, застраховане и осигуряване“ – трима. Теми на дисертационните трудове: 1. „Правно-икономически аспекти на преструктурирането и прекралификацията на човешките ресурси в добивната промишленост“; 2. „Теоретико-методологични аспекти на финансовия мениджмънт в предприятията от добивната промишленост“; 3. „Социално-икономически аспекти при тенденциите в развитието на предприятията от добивната промишленост“. В срок 2 месеца от обнародването в „Държавен вестник“ на обявлението кандидатите подават следните документи на хартиен и електронен носител (CD или флаш памет): 1. заявление (молба) до ректора на Висшето училище по застраховане и финанси за участие в конкурса; 2. автобиография по европейски образец; 3. данни от документ за самоличност; 4. формуляр за съгласие за обработка на лични данни при кандидатстване; 5. оригинал на дипломата за придобитата образователно-квалификационна степен „магистър“ или нотариално заверено нейно копие, или уверение от съответното висше училище за придобитата образователно-квалификационна степен „магистър“; 6. приложение към дипломата за придобитата образователно-квалификационна степен „магистър“ или нотариално заверено негово копие; 7. документи, удостоверяващи нивото на владеене на чужд език (ако има такива), включително сертификати за удостоверяване на нивото; 8. други документи, удостоверяващи интереси и постижения в съответната научна област (ако има такива). След разглеждане на подадените документи от назначена със заповед на ректора комисия и допускане до участието им в обявения конкурс кандидатите полагат конкурсни приемни изпити по специалността и по чужд език (английски език) с отделни оценки. Изпитът за всяка докторска програма задължително съдържа писмен и устен компонент. Писменият изпит е анонимен и с продължителност три астрономически часа. До устен изпит се допускат получилите на писмения изпит оценка най-малко „много добър 4,50“. Успешно положили изпита са кандидатите, които са получили средна оценка от писмения и устния изпит най-малко „много добър 5,00“. До участие в конкурсния изпит по чужд език се допускат само кандидатите за докторанти, положили успешно изпита по специалността. За успешно положили изпита по чужд език се считат кандидатите с оценка не по-ниска от „добър 4,00“. Когато кандидат-докторантите разполагат с международно признат сертификат за владеене на английски език и/или диплома от гимназия/ВУЗ, която удостоверява необходимото ниво чрез съответния хорариум, те могат да не се явяват на конкурсен изпит по чужд език при В1 – добър 4,00; В2 – много добър 5,00; C1 и C2 – отличен 6,00. Докторантите се класират по успех и се зачисляват със заповед на ректора в редовна форма на обучение, като сключват трудов договор с ВУЗФ. Подаване на документи: лично – документи се приемат в отдел „Докторанти“, ет. 4, каб. № 409; с физическата поща на следния адрес: ВУЗФ, София, ул. Гусла № 1.", "id": 8253},
    {"text": "40. – Министърът на енергетиката на основание постъпило заявление за проучване на подземни богатства и във връзка с чл. 39, ал. 1, т. 3 от Закона за подземните богатства съобщава, че предстои да се предостави разрешение за проучване на подземни богатства по чл. 2, ал. 1, т. 5 от Закона за подземните богатства – строителни материали, в площ „Могилата“, разположена в землището на с. Брястовец, община Бургас, област Бургас", "id": 8270},
    {"text": "КОМИСИЯ ЗА ПУБЛИЧЕН НАДЗОР НАД РЕГИСТРИРАНИТЕ ОДИТОРИ Наредба за изменение и допълнение на Наредба № 2 от 7 юли 2020 г. за формата и съдържанието на годишните доклади за дейността на одитните комитети в предприятията от обществен интерес (обн., ДВ, бр. 74 от 2020 г.; изм. и доп., бр. 30 от 2021 г.)§ 1. В чл. 2 думите „Закона за независимия финансов одит“ се заменят със „Закона за независимия финансов одит и изразяването на сигурност по устойчивостта“. § 2. В чл. 4 се правят следните изменения и допълнения: 1. Досегашният текст става ал. 1 и се изменя така: „(1) Докладът съдържа информация в следните области:  1. данни на предприятието; 2. данни за контакт с председателя на одитния комитет;3. данни за състава на одитния комитет; 4. данни за заседанията на одитния комитет и негови срещи с ръководството на предприятието и лицата, натоварени с общото управление; 5. информация относно обсъждане на одитния комитет с регистрирания одитор (одитори) на: планирането на одиторския ангажимент (одиторски ангажименти); текущото изпълнение на одиторския ангажимент (одиторски ангажименти); ключови одиторски въпроси на одиторския ангажимент; изразеното одиторско мнение (направеното заключение); предоставения до одитния комитет допълнителен доклад от одитора (одиторите); 6. информация за процедурата по подбор на регистриран одитор (регистрирани одитори); 7. данни за избрания регистриран одитор (регистрирани одитори); 8. информация за дадени от одитния комитет одобрения (отказ от одобрение) за извършване на допълнителни услуги от регистрирания одитор (одитори); 9. наличие на установени от одитния комитет заплахи за независимостта на регистрирания одитор (одитори) и за препоръчани предпазни мерки; 10. информация за препоръки от одитния комитет към ръководството на предприятието за: подобряване на системата за финансово отчитане в предприятието; подобряване на системите за вътрешен контрол и управление на риска в предприятието; подобряване на функцията на вътрешния одит в предприятието; 11. информация за основни затруднения при изпълнението на функциите на одитния комитет; 12. наличие на несъгласия между одитния комитет и регистрирания одитор (одитори) относно одиторския доклад; 13. мнение на одитния комитет за работата на екипа на регистрирания одитор (регистрираните одитори) по ангажимента.“ 2. Създават се ал. 2 и 3: „(2) Към доклада се прилага копие на допълнителния доклад до одитния комитет, издаден на основание чл. 60 от Закона за независимия финансов одит и изразяването на сигурност по устойчивостта. (3) Когато е приложимо, информацията в доклада се отнася както за регистрирания одитор (одитори), изпълняващ ангажимент за задължителен финансов одит, така и за регистрирания одитор (одитори), изпълняващ задължителен ангажимент за сигурност по устойчивостта.“ § 3. В чл. 5, ал. 1 думите „попълване на въпроси“ се заменят с „подаване на съответната информация“. § 4. В чл. 8, ал. 1 думите „създаден за целта“ и „съгласно приложение № 2“ се заличават. § 5. В чл. 9 ал. 3 се отменя. § 6. В параграф 1 от преходните и заключителните разпоредби думите „Закона за независимия финансов одит“ се заменят със „Закона за независимия финансов одит и изразяването на сигурност по устойчивостта“. § 7. Приложение № 1 към чл. 3 и приложение № 2 към чл. 6 се заличават. Председател: Огнян Симеонов", "id": 8284},
]

In [ ]:
#@title ### Use langchain to define prompt template with context

# Extract the first document's text
first_document_text = documents[0]["text"]

# Define the prompt template, incorporating the first document's text
prompt_template = """
Ти си експерт с юридическо образование и познания в публичния сектор. Целта ти е да представиш информацията кратко и по-възможно най-разбираем начин.

First Document: {first_document}

Retrieved Documents:
{context}

Query:
{query}
"""

# Set up the prompt with the first document included
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["first_document", "context", "query"]
)

# Example query and context
query = "Какво се решава с постановление 493?"
retrieved_context = "\n".join([doc["text"] for doc in documents])

# Format the prompt
formatted_prompt = prompt.format(
    first_document=first_document_text,
    context=retrieved_context,
    query=query
)

# Print the formatted prompt
print("Formatted Prompt:")
print(formatted_prompt)


Formatted Prompt:

Ти си експерт с юридическо образование и познания в публичния сектор. Целта ти е да представиш информацията кратко и по-възможно най-разбираем начин.

First Document: ПОСТАНОВЛЕНИЕ № 493 ОТ 30 ДЕКЕМВРИ 2024 Г. за одобряване на допълнителни разходи по бюджета на Министерството на енергетиката за 2024 г. МИНИСТЕРСКИЯТ СЪВЕТ ПОСТАНОВИ: Чл. 1. (1) Одобрява допълнителни разходи в размер до 2 200 000 лв. по бюджета на Министерството на енергетиката за 2024 г. за финансова подкрепа на крайните битови клиенти на електрическа енергия, останали без електрозахранване в периода от 21 декември 2024 г. – 31 декември 2024 г. включително.  (2) Средствата по ал. 1 да се осигурят чрез преструктуриране на разходите и/или трансферите по централния бюджет за 2024 г. Чл. 2. (1) Със сумата по чл. 1, ал. 1 да се увеличат разходите по бюджета на Министерството на енергетиката за 2024 г. по „Политика в областта на устойчивото и конкурентоспособно енергийно развитие“, бюджетна програма „Сигурн

In [ ]:
# @title #### Get response from Gemini with additional context, not relevant for the question.
response = model.generate_content(formatted_prompt)
print(response.text)

Постановление 493 одобрява допълнителни разходи до 2 200 000 лв. за финансова подкрепа на крайните битови клиенти на електрическа енергия, останали без електрозахранване от 21 декември 2024 г. до 31 декември 2024 г. включително.


BgGPT didn't give a response when the full query with context was submitted.

## Experiment with multilingual universal sentence encoder for document retrieval

In [ ]:
# @title #### Load documents as list
documents = [
    "ПОСТАНОВЛЕНИЕ № 493 ОТ 30 ДЕКЕМВРИ 2024 Г. за одобряване на допълнителни разходи по бюджета на Министерството на енергетиката за 2024 г. МИНИСТЕРСКИЯТ СЪВЕТ ПОСТАНОВИ: Чл. 1. (1) Одобрява допълнителни разходи в размер до 2 200 000 лв. по бюджета на Министерството на енергетиката за 2024 г. за финансова подкрепа на крайните битови клиенти на електрическа енергия, останали без електрозахранване в периода от 21 декември 2024 г. – 31 декември 2024 г. включително.  (2) Средствата по ал. 1 да се осигурят чрез преструктуриране на разходите и/или трансферите по централния бюджет за 2024 г. Чл. 2. (1) Със сумата по чл. 1, ал. 1 да се увеличат разходите по бюджета на Министерството на енергетиката за 2024 г. по „Политика в областта на устойчивото и конкурентоспособно енергийно развитие“, бюджетна програма „Сигурност при енергоснабдяването и при управлението на РАО и ИЕЯС“. (2) Със сумата по чл. 1, ал. 1 да се увеличат показателите по чл. 21, ал. 3 от Закона за държавния бюджет на Република България за 2024 г. (3) Икономията по показателите по ал. 2 не може да се използва за поемането на нови ангажименти и нови задължения за разходи за други цели.Чл. 3. (1) За извършване на плащанията по чл. 1, ал. 1 се използва сметка за чужди средства на Министерството на енергетиката.(2) Преведените до 31 декември 2024 г. по сметката по ал. 1 суми се отразяват в изпълнението на съответните разходни показатели по бюджета на Министерството на енергетиката за 2024 г. Чл. 4. Министърът на енергетиката да извърши съответните промени по бюджета на Министерството на енергетиката за 2024 г. и да уведоми министъра на финансите. Чл. 5. Министърът на финансите да извърши съответните промени по централния бюджет за 2024 г. Чл. 6. Възлага на министъра на енергетиката и на министъра на финансите да утвърдят указания за изплащане на целевата финансова подкрепа по чл. 1.",
    "621. – Висшето училище по застраховане и финанси – София, обявява конкурс за прием на докторанти в редовна форма на обучение по чл. 21, ал. 7 от ЗВО (докторските места се финансират от Програма „Образование“ по проект BG05SFPR001-3.004-0002-C01 „Развитие и усъвършенстване на професионалното и интердисциплинарно образование в техническите и икономически науки и повишаване на научноизследователския капацитет на добивната промишленост чрез проектна докторантура“) в ПН 3.8. Икономика, специалност: „Финанси, застраховане и осигуряване“ – трима. Теми на дисертационните трудове: 1. „Правно-икономически аспекти на преструктурирането и прекралификацията на човешките ресурси в добивната промишленост“; 2. „Теоретико-методологични аспекти на финансовия мениджмънт в предприятията от добивната промишленост“; 3. „Социално-икономически аспекти при тенденциите в развитието на предприятията от добивната промишленост“. В срок 2 месеца от обнародването в „Държавен вестник“ на обявлението кандидатите подават следните документи на хартиен и електронен носител (CD или флаш памет): 1. заявление (молба) до ректора на Висшето училище по застраховане и финанси за участие в конкурса; 2. автобиография по европейски образец; 3. данни от документ за самоличност; 4. формуляр за съгласие за обработка на лични данни при кандидатстване; 5. оригинал на дипломата за придобитата образователно-квалификационна степен „магистър“ или нотариално заверено нейно копие, или уверение от съответното висше училище за придобитата образователно-квалификационна степен „магистър“; 6. приложение към дипломата за придобитата образователно-квалификационна степен „магистър“ или нотариално заверено негово копие; 7. документи, удостоверяващи нивото на владеене на чужд език (ако има такива), включително сертификати за удостоверяване на нивото; 8. други документи, удостоверяващи интереси и постижения в съответната научна област (ако има такива). След разглеждане на подадените документи от назначена със заповед на ректора комисия и допускане до участието им в обявения конкурс кандидатите полагат конкурсни приемни изпити по специалността и по чужд език (английски език) с отделни оценки. Изпитът за всяка докторска програма задължително съдържа писмен и устен компонент. Писменият изпит е анонимен и с продължителност три астрономически часа. До устен изпит се допускат получилите на писмения изпит оценка най-малко „много добър 4,50“. Успешно положили изпита са кандидатите, които са получили средна оценка от писмения и устния изпит най-малко „много добър 5,00“. До участие в конкурсния изпит по чужд език се допускат само кандидатите за докторанти, положили успешно изпита по специалността. За успешно положили изпита по чужд език се считат кандидатите с оценка не по-ниска от „добър 4,00“. Когато кандидат-докторантите разполагат с международно признат сертификат за владеене на английски език и/или диплома от гимназия/ВУЗ, която удостоверява необходимото ниво чрез съответния хорариум, те могат да не се явяват на конкурсен изпит по чужд език при В1 – добър 4,00; В2 – много добър 5,00; C1 и C2 – отличен 6,00. Докторантите се класират по успех и се зачисляват със заповед на ректора в редовна форма на обучение, като сключват трудов договор с ВУЗФ. Подаване на документи: лично – документи се приемат в отдел „Докторанти“, ет. 4, каб. № 409; с физическата поща на следния адрес: ВУЗФ, София, ул. Гусла № 1.",
    "40. – Министърът на енергетиката на основание постъпило заявление за проучване на подземни богатства и във връзка с чл. 39, ал. 1, т. 3 от Закона за подземните богатства съобщава, че предстои да се предостави разрешение за проучване на подземни богатства по чл. 2, ал. 1, т. 5 от Закона за подземните богатства – строителни материали, в площ „Могилата“, разположена в землището на с. Брястовец, община Бургас, област Бургас",
    "КОМИСИЯ ЗА ПУБЛИЧЕН НАДЗОР НАД РЕГИСТРИРАНИТЕ ОДИТОРИ Наредба за изменение и допълнение на Наредба № 2 от 7 юли 2020 г. за формата и съдържанието на годишните доклади за дейността на одитните комитети в предприятията от обществен интерес (обн., ДВ, бр. 74 от 2020 г.; изм. и доп., бр. 30 от 2021 г.)§ 1. В чл. 2 думите „Закона за независимия финансов одит“ се заменят със „Закона за независимия финансов одит и изразяването на сигурност по устойчивостта“. § 2. В чл. 4 се правят следните изменения и допълнения: 1. Досегашният текст става ал. 1 и се изменя така: „(1) Докладът съдържа информация в следните области:  1. данни на предприятието; 2. данни за контакт с председателя на одитния комитет;3. данни за състава на одитния комитет; 4. данни за заседанията на одитния комитет и негови срещи с ръководството на предприятието и лицата, натоварени с общото управление; 5. информация относно обсъждане на одитния комитет с регистрирания одитор (одитори) на: планирането на одиторския ангажимент (одиторски ангажименти); текущото изпълнение на одиторския ангажимент (одиторски ангажименти); ключови одиторски въпроси на одиторския ангажимент; изразеното одиторско мнение (направеното заключение); предоставения до одитния комитет допълнителен доклад от одитора (одиторите); 6. информация за процедурата по подбор на регистриран одитор (регистрирани одитори); 7. данни за избрания регистриран одитор (регистрирани одитори); 8. информация за дадени от одитния комитет одобрения (отказ от одобрение) за извършване на допълнителни услуги от регистрирания одитор (одитори); 9. наличие на установени от одитния комитет заплахи за независимостта на регистрирания одитор (одитори) и за препоръчани предпазни мерки; 10. информация за препоръки от одитния комитет към ръководството на предприятието за: подобряване на системата за финансово отчитане в предприятието; подобряване на системите за вътрешен контрол и управление на риска в предприятието; подобряване на функцията на вътрешния одит в предприятието; 11. информация за основни затруднения при изпълнението на функциите на одитния комитет; 12. наличие на несъгласия между одитния комитет и регистрирания одитор (одитори) относно одиторския доклад; 13. мнение на одитния комитет за работата на екипа на регистрирания одитор (регистрираните одитори) по ангажимента.“ 2. Създават се ал. 2 и 3: „(2) Към доклада се прилага копие на допълнителния доклад до одитния комитет, издаден на основание чл. 60 от Закона за независимия финансов одит и изразяването на сигурност по устойчивостта. (3) Когато е приложимо, информацията в доклада се отнася както за регистрирания одитор (одитори), изпълняващ ангажимент за задължителен финансов одит, така и за регистрирания одитор (одитори), изпълняващ задължителен ангажимент за сигурност по устойчивостта.“ § 3. В чл. 5, ал. 1 думите „попълване на въпроси“ се заменят с „подаване на съответната информация“. § 4. В чл. 8, ал. 1 думите „създаден за целта“ и „съгласно приложение № 2“ се заличават. § 5. В чл. 9 ал. 3 се отменя. § 6. В параграф 1 от преходните и заключителните разпоредби думите „Закона за независимия финансов одит“ се заменят със „Закона за независимия финансов одит и изразяването на сигурност по устойчивостта“. § 7. Приложение № 1 към чл. 3 и приложение № 2 към чл. 6 се заличават. Председател: Огнян Симеонов"
]

In [ ]:
# @title #### Load a multilingual encoder and create documents embeddings
model_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
embedding_model = hub.load(model_url)

# Test embedding generation
embeddings = embedding_model(documents)
print("Embeddings shape:", embeddings.shape)


Embeddings shape: (4, 512)


In [ ]:
# @title #### Compute similarity matrix for documents
similarity_matrix = cosine_similarity(embeddings)

print("Cosine Similarity Matrix:")
print(similarity_matrix)


Cosine Similarity Matrix:
[[1.0000002  0.30162165 0.37762737 0.3370813 ]
 [0.30162165 1.0000002  0.24172184 0.41692984]
 [0.37762737 0.24172184 0.9999997  0.17870328]
 [0.3370813  0.41692984 0.17870328 1.        ]]


In [ ]:
#@title ####Experiment with mUSE with english and pre-defined responses

# Predefined sentences
sentences = [
    "5G technology improves bandwidth and reduces latency.",
    "Roaming charges apply to international calls.",
    "Data plans include unlimited calls within the network.",
    "VoLTE enables high-quality voice calls over LTE networks."
]

# Generate embeddings for predefined sentences
sentence_embeddings = embedding_model(sentences)

try:
    while True:
        # Take user input for the query
        query = input("Enter your query (or type 'exit' to quit): ").strip()
        if query.lower() == 'exit':
            print("Exiting the program. Goodbye!")
            break

        # Generate embedding for the query
        query_embedding = embedding_model([query])

        # Compute cosine similarity
        similarities = cosine_similarity(query_embedding, sentence_embeddings)[0]  # Get the first row

        # Find the most similar sentence
        most_similar_idx = np.argmax(similarities)  # Index of the most similar sentence
        most_similar_sentence = sentences[most_similar_idx]
        most_similar_score = similarities[most_similar_idx]

        # Print the results
        print("\nQuery:", query)
        print("Most Similar Sentence:", most_similar_sentence)
        print(f"Similarity Score: {most_similar_score:.2f}\n")

except KeyboardInterrupt:
    print("\n\nProgram interrupted by user.")


Enter your query (or type 'exit' to quit): What are the advantages of 5G?

Query: What are the advantages of 5G?
Most Similar Sentence: 5G technology improves bandwidth and reduces latency.
Similarity Score: 0.48

Enter your query (or type 'exit' to quit): Do data plans include unlimited calls in the network?

Query: Do data plans include unlimited calls in the network?
Most Similar Sentence: Data plans include unlimited calls within the network.
Similarity Score: 0.77



Program interrupted by user.


In [ ]:
#@title #### Find most similar document to example query in Bulgarian

# Generate embeddings for predefined sentences
sentence_embeddings = embedding_model(documents)

while True:
    # Take user input for the query
    query = input("Enter your query (or type 'exit' to quit): ")
    if query.lower() == 'exit':
        print("Exiting the program.")
        break

    # Generate embedding for the query
    query_embedding = embedding_model([query])

    # Compute cosine similarity
    similarities = cosine_similarity(query_embedding, sentence_embeddings)[0]  # Get the first row

    # Find the most similar sentence
    most_similar_idx = np.argmax(similarities)  # Index of the most similar sentence
    most_similar_sentence = documents[most_similar_idx]
    most_similar_score = similarities[most_similar_idx]

    # Print the results
    print("\nQuery:", query)
    print("Most Similar Sentence:", most_similar_sentence)
    print(f"Similarity Score: {most_similar_score:.2f}\n")

Enter your query (or type 'exit' to quit): Какви нови разпоредби има в сферата на енергетиката?

Query: Какви нови разпоредби има в сферата на енергетиката?
Most Similar Sentence: ПОСТАНОВЛЕНИЕ № 493 ОТ 30 ДЕКЕМВРИ 2024 Г. за одобряване на допълнителни разходи по бюджета на Министерството на енергетиката за 2024 г. МИНИСТЕРСКИЯТ СЪВЕТ ПОСТАНОВИ: Чл. 1. (1) Одобрява допълнителни разходи в размер до 2 200 000 лв. по бюджета на Министерството на енергетиката за 2024 г. за финансова подкрепа на крайните битови клиенти на електрическа енергия, останали без електрозахранване в периода от 21 декември 2024 г. – 31 декември 2024 г. включително.  (2) Средствата по ал. 1 да се осигурят чрез преструктуриране на разходите и/или трансферите по централния бюджет за 2024 г. Чл. 2. (1) Със сумата по чл. 1, ал. 1 да се увеличат разходите по бюджета на Министерството на енергетиката за 2024 г. по „Политика в областта на устойчивото и конкурентоспособно енергийно развитие“, бюджетна програма „Сигурност пр

## Next steps

*   Combine retrieval and summarisation
*   Compare outputs from different models and prompts

